In [1]:
from datetime import datetime, timedelta
import mysql.connector

# Define the start and end dates
start_date = datetime(2018, 1, 1)
end_date = datetime.today()

# Create a list of dates
date_list = []

# Iterate from start_date to end_date and add each date to the list
current_date = start_date
while current_date <= end_date:
    date_list.append((current_date,))
    current_date += timedelta(days=1)

In [2]:
host = "finance.cvm2aaxkmu43.us-east-2.rds.amazonaws.com"
user = "admin"
password = "Nk7f4mJr6?A"
database = "finance"

conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = conn.cursor()

### Insert NU price before IPO

In [3]:
nu_prices = [
        ("NU", "2020-04-01", 1.7354,  1.7354),
        ("NU", "2020-07-01", 1.7883,  1.7883),
        ("NU", "2020-10-01", 1.7883,  1.7883),
        ("NU", "2021-01-01", 3.9783,  3.9783),
        ("NU", "2021-04-01", 3.9783,  3.9783),
        ("NU", "2021-07-01", 4.5283,  4.5283),
        ("NU", "2021-10-01", 4.5283,  4.5283)]

In [4]:
cursor.executemany(
    """
    INSERT IGNORE INTO asset_price(ticker, quote_date, open_price, close_price)
    VALUES(%s, %s, %s, %s)
    """, nu_prices)

### Insert dates

In [5]:
cursor.executemany(
    "INSERT IGNORE INTO dates(date) VALUES(%s)",
    date_list
)

### Insert daily_asset_price

In [6]:
cursor.execute(
    """
    INSERT IGNORE INTO daily_asset_price(ticker, date, price)
    WITH date_range AS
    (
        SELECT
            ticker,
            quote_date,
            LEAD(quote_date) OVER(PARTITION BY ticker ORDER BY quote_date) as next_date,
            close_price
        FROM asset_price ap
    )
    SELECT
        ticker,
        COALESCE(d.date, quote_date) as date,
        close_price as price
    FROM date_range dr
    LEFT JOIN dates d
    ON d.date >= dr.quote_date AND d.date < dr.next_date
    """
)

### Insert variable_income_daily_balance

In [7]:
cursor.execute(
    """
    INSERT IGNORE INTO variable_income_daily_balance
    WITH dolar AS(
        SELECT date, price as dolar_price
        FROM daily_asset_price
        WHERE ticker = "BRL=X"
    ),
    currency AS (
        SELECT ticker, MAX(currency) as currency
        FROM variable_income_operations
        GROUP BY ticker
    ),
    operation AS (
        SELECT 	
            ticker,
            currency,
            operation_date,
            LEAD(operation_date) OVER (PARTITION BY ticker ORDER BY operation_date) AS next_operation_date,
            SUM(CASE WHEN operation_type = "buy" THEN amount else amount * -1 END) AS amount
        FROM variable_income_operations o
        GROUP BY ticker, currency, operation_date, operation_type
    ),
    balance AS (
        SELECT
            p.ticker,
            p.date,
            p.price,
            dolar_price,
            c.currency, 
            amount as amount_change,
            SUM(amount) OVER(PARTITION BY ticker ORDER BY p.date) AS amount
        FROM daily_asset_price p
        LEFT JOIN operation o
        ON p.ticker = o.ticker AND p.date = o.operation_date
        LEFT JOIN dolar d
        ON d.date = p.date
        INNER JOIN currency c
        ON c.ticker = p.ticker 
    )
    SELECT
        ticker,
        date,
        price,
        dolar_price,
        currency,
        amount_change,
        amount,
        CASE WHEN currency = "dolar" THEN (price * dolar_price) * amount ELSE price * amount END as value
    FROM balance
    WHERE amount <> 0;
    """
)

In [8]:
conn.commit()

In [9]:
conn.close()